In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd'/content/drive/My Drive/ML'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/ML


In [6]:
import pickle #used for binary files
import numpy #used for reshape network input
from music21 import converter, instrument, note, stream, chord, duration #used for midi files processing
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

timestep = 0.25
sequence_length = int(8 / timestep)

def generator_process():
    # Open notes file
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab) # sending the network output not normilized because firts it select a random sequence as a starting point and then normalize that output before send it for prediction
    create_midi(prediction_output)
    print("Done!")


def prepare_sequences(notes, pitchnames, n_vocab):
    # Mapping notes to integer-based data. Create a dictionary to map pitches to integers
    note_to_int = dict((note, number+1) for number, note in enumerate(pitchnames)) # +1 to consider rest notes
    note_to_int["NULL"] = 0 # this is for rests times

    # Create input sequences.
    network_input = []
    #output = []
    for i in range(0, len(notes) - sequence_length, 1): #the limit is because it won't be an output if there are sequence_length elements reamining
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in]) #collects all input sequences but in the integer form. list of n lists with sequence_length elements in each one
        #output.append(note_to_int[sequence_out])  #collects all output sequences but in the integer form
    
    n_patterns = len(network_input)
 
    # Reshape the input into a format compatible with LSTM layers. From horizontal to vertical form
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # Normalize input. Get values from 0 to 1 representing our "classes" into a range from 0 to 1
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.4,
        return_sequences=True
    ))
    model.add(Dropout(0.4))
    model.add(LSTM(512))
    #model.add(Dropout(0.5))
    #model.add(LSTM(512)) #number of nodes
    #model.add(Dense(256)) #number of nodes
    #model.add(Dropout(0.5))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax')) 
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    model.summary()

    # Load the weights to each node
    model.load_weights('weights-improvement-19-3.7358-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    int_to_note = dict((number + 1, note) for number, note in enumerate(pitchnames))
    int_to_note[0] = "NULL"
    
    # Function needed to get different pattern for each verse, chorus and bridge
    def get_start():
        # Pick a random sequence from the input as a starting point for the prediction
        start = numpy.random.randint(0, len(network_input) - 1)
        pattern = network_input[start]
        prediction_output = []
        return pattern, prediction_output

    # generate verse 1
    verse1_pattern, verse1_prediction_output = get_start()
    print("Starting movement 1...")
    for note_index in range(4 * sequence_length):
        # reshape our random sequence
        prediction_input = numpy.reshape(verse1_pattern, (1, len(verse1_pattern), 1))
        # normalize
        prediction_input = prediction_input / float(n_vocab)
        # predict note by note
        prediction = model.predict(prediction_input, verbose=0)
        
        index = numpy.argmax(prediction)
        #print("Index "+ str(index))
        result = int_to_note[index]
        verse1_prediction_output.append(result)

        verse1_pattern.append(index)
        verse1_pattern = verse1_pattern[1 : len(verse1_pattern)]
    
    # generate verse 2
    verse2_pattern = verse1_pattern
    verse2_prediction_output = []
    print("Starting movement 2...")
    for note_index in range(4 * sequence_length):
        # reshape our random sequence
        prediction_input = numpy.reshape(verse2_pattern, (1, len(verse2_pattern), 1))
        # normalize
        prediction_input = prediction_input / float(n_vocab)
        # predict note by note
        prediction = model.predict(prediction_input, verbose=0)
        
        index = numpy.argmax(prediction)
        #print("Index "+ str(index))
        result = int_to_note[index]
        verse2_prediction_output.append(result)

        verse2_pattern.append(index)
        verse2_pattern = verse2_pattern[1 : len(verse2_pattern)]
    
    print("Generation completed!")
    return(verse1_prediction_output + verse2_prediction_output)

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    print("Saving file...")
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        if "$" in pattern:
            pattern, dur = pattern.split("$")
            if "/" in dur:
                a, b = dur.split("/")
                dur = float(a) / float(b)
            else:
                dur = float(dur)
  
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_chord.duration = duration.Duration(dur)
            output_notes.append(new_chord)
        # pattern is a rest
        elif pattern is "NULL":
            offset += 0.25        
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            new_note.duration = duration.Duration(dur)
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.25

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')

if __name__ == "__main__":
    generator_process()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 32, 512)           1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 512)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_3 (Dense)              (None, 3689)              1892457   
_________________________________________________________________
activation_3 (Activation)    (None, 3689)              0         
Total params: 5,044,329
Trainable params: 5,044,329
Non-trainable params: 0
_________________________________________________________________
Starting movement 1...
Starting movement 2...
Generation completed!
Saving file...
Done!
